In [74]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))

from src.data_prep.preprocessors import *
%load_ext autoreload
%autoreload 2

import json
import polars as pl
import warnings
warnings.simplefilter('ignore')

pl.Config.set_tbl_cols(100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


polars.config.Config

In [ ]:
# df.select([col for col in df.collect_schema().names() if 'price' in col]).sample(10, seed=42)
# data_processed.select([col for col, dtype in data_processed.schema.items() if dtype == pl.Utf8]).sample(10, seed=42)
# data_processed.select([col for col in data_processed.collect_schema().names() if 'DATE' in col])

In [2]:
"""
Clasificación de artículos nuevos/usados para MercadoLibre
"""

def build_dataset():
    """Carga y prepara los datos de entrenamiento y prueba"""
    data_raw = [json.loads(line) for line in open('../../data/raw/MLA_100k_checked_v3.jsonlines')]
    return data_raw

if __name__ == "__main__":
    print("Cargando dataset...")
    # Convención de nomenclatura de sklearn:
    # - X_train/X_test: Lista de diccionarios con features
    # - y_train/y_test: Etiquetas ('new' o 'used')
    data_raw = build_dataset()
    
    # ------------------------------------------
    # Insertar código de ML aquí:
    # - Preprocesamiento
    # - Entrenamiento del modelo
    # - Evaluación
    # ------------------------------------------

Cargando dataset...


In [80]:
# Preprocesadores
from sklearn.pipeline import Pipeline

# Definir la pipeline (asegúrate de que todos los transformadores trabajen con LazyFrame)
pipe = Pipeline([
    ('raw_data', RawDataPreprocessor()),
    ('rename_columns', ColumnsRenameTransformer()),
    ('date_columns', DateColumnsTransformer(patterns=['time', 'date'])),
    ('duplicated_columns', DropDuplicateColumnsTransformer()),
    ('duplicated_rows', DropDuplicatedRowsTransformer()),
    ('nan_values', FillMissingValuesTransformer()),
    ('drop_columns', DropColumnsTransformer(0.2)),
    ('unnecessary_columns', DropUnnecessaryColumnsTransformer()),
    ('categorical_columns', CategoricalColumnsTransformer()),
    ('numeric_columns', NumericColumnTransformer()),
])

# Aplicar la pipeline a los datos y materializar el resultado (en este caso usando GPU si es posible)
data_processed = pipe.fit_transform(data_raw).collect()

# Contar registros después de la transformación
row_count_after = data_processed.select(pl.count("*"))[0, 0]

# Contar columnas después de la transformación (ahora data_processed es un DataFrame eager)
col_count_after = len(data_processed.columns)

print(f'Número de registros después de la transformación: {row_count_after}')
print(f'Número de columnas después de la transformación: {col_count_after}')

# Supongamos que 'df' es tu DataFrame de Polars
missing_percentages = (
    data_processed.select([pl.col(c).is_null().mean().alias(c) for c in data_processed.collect_schema().names()])
      .unpivot()  # Convierte el DataFrame de una sola fila a dos columnas: 'variable' y 'value'
      .sort("value", descending=True)  # Ordena de mayor a menor según el porcentaje de nulos
)

print(missing_percentages)

Número de registros después de la transformación: 1459114
Número de columnas después de la transformación: 30
shape: (30, 2)
┌───────────────────────────┬──────────┐
│ variable                  ┆ value    │
│ ---                       ┆ ---      │
│ str                       ┆ f64      │
╞═══════════════════════════╪══════════╡
│ attr_attribute_group_name ┆ 0.174394 │
│ attr_attribute_group_id   ┆ 0.174298 │
│ attr_name                 ┆ 0.174298 │
│ attr_id                   ┆ 0.174298 │
│ seller_address_city       ┆ 0.000669 │
│ …                         ┆ …        │
│ status                    ┆ 0.0      │
│ initial_quantity          ┆ 0.0      │
│ start_time                ┆ 0.0      │
│ sold_quantity             ┆ 0.0      │
│ available_quantity        ┆ 0.0      │
└───────────────────────────┴──────────┘


In [82]:
df_product_dup = data_processed.filter(pl.col('product_id').is_duplicated())
print('Filas duplicadas en product_id:')
print(len(df_product_dup))

Filas duplicadas en product_id:
1427588


In [84]:
# Conteo de productos únicos
data_processed.group_by('product_id', 'title').agg(pl.count('product_id').alias('count')).sort('count', descending=True)

product_id,title,count
str,str,u32
"""mla6356328214""","""remeras super héroes / marvel …",7616
"""mla1470121909""","""chelsea market jean mujer chup…",5346
"""mla1807504057""","""zapatillas sneakers plataforma…",5184
"""mla6655124735""","""campera nena modal estampada b…",4536
"""mla2618837555""","""pantuflas garras mujer-niñas m…",3888
…,…,…
"""mla8638365276""","""mujeres benefactoras del mundo…",1
"""mla7548005604""","""cuna funcional de melamina""",1
"""mla5273913810""","""super colección corto maltés n…",1


In [81]:
data_processed[0]

seller_address_country,seller_address_state,seller_address_city,condition,base_price,shipping_local_pick_up,shipping_free_shipping,shipping_mode,seller_id,site_id,listing_type_id,attr_attribute_group_id,attr_name,attr_attribute_group_name,attr_id,buying_mode,category_id,last_updated,product_id,accepts_mercadopago,currency_id,title,automatic_relist,date_created,stop_time,status,initial_quantity,start_time,sold_quantity,available_quantity
str,str,str,str,f64,bool,bool,str,i64,str,str,str,str,str,str,str,str,datetime[ms],str,bool,str,str,bool,datetime[ms],datetime[ms],str,i64,datetime[ms],i64,i64
"""argentina""","""buenos aires""","""caseros""","""used""",44900.0,false,false,"""not_specified""",4421644297,"""mla""","""silver""","""confort""","""asientos eléctricos""","""confort""","""mla1744-asienelec""","""classified""","""mla24282""",2015-10-09 15:55:48,"""mla5192131886""",false,"""ars""","""volkswagen golf 2.0 2011 dado …",false,2015-08-09 10:37:48,2015-11-10 15:48:59.901,"""active""",1,2015-08-09 10:37:47,0,1


In [85]:
# Enviar datos provisionales
data_processed.write_parquet('../../data/interim/data_interim.parquet')

In [4]:
# Esquema original
df = pl.DataFrame(data_raw, infer_schema_length=len(data_raw))
df.schema

Schema([('seller_address',
         Struct({'country': Struct({'name': String, 'id': String}), 'state': Struct({'name': String, 'id': String}), 'city': Struct({'name': String, 'id': String})})),
        ('warranty', String),
        ('sub_status', List(String)),
        ('condition', String),
        ('deal_ids', List(String)),
        ('base_price', Float64),
        ('shipping',
         Struct({'local_pick_up': Boolean, 'methods': List(Null), 'tags': List(String), 'free_shipping': Boolean, 'mode': String, 'dimensions': String, 'free_methods': List(Struct({'rule': Struct({'value': Null, 'free_mode': String}), 'id': Int64}))})),
        ('non_mercado_pago_payment_methods',
         List(Struct({'description': String, 'id': String, 'type': String}))),
        ('seller_id', Int64),
        ('variations',
         List(Struct({'attribute_combinations': List(Struct({'value_id': String, 'name': String, 'value_name': String, 'id': String})), 'seller_custom_field': String, 'picture_ids': Lis

In [4]:
# Columnas anidadas
df = pl.DataFrame(data_raw, infer_schema_length=len(data_raw))
df = df.select([col for col, dtype in df.schema.items() if dtype == pl.List])
df.schema

Schema([('sub_status', List(String)),
        ('deal_ids', List(String)),
        ('non_mercado_pago_payment_methods',
         List(Struct({'description': String, 'id': String, 'type': String}))),
        ('variations',
         List(Struct({'attribute_combinations': List(Struct({'value_id': String, 'name': String, 'value_name': String, 'id': String})), 'seller_custom_field': String, 'picture_ids': List(String), 'sold_quantity': Int64, 'available_quantity': Int64, 'id': Int64, 'price': Float64}))),
        ('attributes',
         List(Struct({'value_id': String, 'attribute_group_id': String, 'name': String, 'value_name': String, 'attribute_group_name': String, 'id': String}))),
        ('tags', List(String)),
        ('coverage_areas', List(Null)),
        ('descriptions', List(String)),
        ('pictures',
         List(Struct({'size': String, 'secure_url': String, 'max_size': String, 'url': String, 'quality': String, 'id': String})))])

In [5]:
# Columnas "complejas"
df.columns

['sub_status',
 'deal_ids',
 'non_mercado_pago_payment_methods',
 'variations',
 'attributes',
 'tags',
 'coverage_areas',
 'descriptions',
 'pictures']

In [3]:
# Asumiendo que "data_raw" es tu lista de registros
df = pl.DataFrame(data_raw, infer_schema_length=len(data_raw)).rename({'id': 'product_id'})

# ================================
# 1. Columnas con listas de strings simples
list_string_columns = ['sub_status', 'deal_ids', 'tags', 'descriptions']
for col in list_string_columns:
    # Une los elementos de la lista en una cadena separada por comas.
    df = df.with_columns(
        pl.col(col).list.join(',').alias(col)
    )

# ================================
# 2. Columnas con listas de estructuras (tipo List(Struct))
complex_cols = [
    ('non_mercado_pago_payment_methods', 'nmp_pm'),
    ('variations', 'var'),
    ('attributes', 'attr'),
    ('pictures', 'pic')
]
for original_col, prefix in complex_cols:
    # Accedemos a los nombres de los campos del struct interno
    current_field_names = [field.name for field in df.schema[original_col].inner.fields]
    # Generamos nuevos nombres usando el prefijo (por ejemplo, "nmp_pm_id" en vez de "id")
    new_field_names = [f'{prefix}_{name}' for name in current_field_names]
    
    # Renombrar internamente cada elemento (struct) de la lista usando list.eval
    df = df.with_columns(
        pl.col(original_col).list.eval(
            pl.element().struct.rename_fields(new_field_names),
            parallel=True
        ).alias(original_col)
    )
    
    # Explota y desanida la columna ya renombrada
    df = df.explode(original_col).unnest(original_col)

# ================================
# 3. Columnas con estructuras anidadas (tipo Struct)
# Estas columnas ya son dict (struct) y al unnest pueden generar nombres conflictivos.
struct_cols = ['seller_address', 'shipping']
for col in struct_cols:
    # Obtener los nombres actuales de los campos del struct
    current_field_names = [field.name for field in df.schema[col].fields]
    # Generar nuevos nombres agregando el nombre de la columna como prefijo
    new_field_names = [f"{col}_{name}" for name in current_field_names]
    # Renombrar internamente los campos del struct
    df = df.with_columns(
        pl.col(col).struct.rename_fields(new_field_names).alias(col)
    )
    # Unnest para pasar los campos a columnas
    df = df.unnest(col)

# Para seller_address, extraemos el campo "name" de cada subcolumna para obtener un valor string
seller_address_cols = ['seller_address_country', 'seller_address_state', 'seller_address_city']
for col in seller_address_cols:
    # Se extrae el campo "name" del struct
    df = df.with_columns(
        pl.col(col).struct.field('name').alias(col)
    )

# ================================
# 4. Columnas especiales
# Procesar shipping.free_methods (si existe)
if 'shipping_free_methods' in df.columns:
    df = df.explode('shipping_free_methods').unnest('shipping_free_methods')
    # Si existe la subestructura 'rule', desempaquetarla y renombrar sus campos
    if 'rule' in df.columns:
        df = df.unnest('rule').rename({
            'value': 'rule_value',
            'free_mode': 'rule_free_mode'
        })

# Procesar variations.attribute_combinations (si existe)
if 'var_attribute_combinations' in df.columns:
    prefix = 'var_comb'
    current_field_names = [field.name for field in df.schema['var_attribute_combinations'].inner.fields]
    new_field_names = [f'{prefix}_{name}' for name in current_field_names]
    df = df.with_columns(
         pl.col('var_attribute_combinations').list.eval(
             pl.element().struct.rename_fields(new_field_names),
             parallel=True
         ).alias('var_attribute_combinations')
    )
    df = df.explode('var_attribute_combinations').unnest('var_attribute_combinations')


In [7]:
df.collect_schema()

Schema([('seller_address_country', String),
        ('seller_address_state', String),
        ('seller_address_city', String),
        ('warranty', String),
        ('sub_status', String),
        ('condition', String),
        ('deal_ids', String),
        ('base_price', Float64),
        ('shipping_local_pick_up', Boolean),
        ('shipping_methods', List(Null)),
        ('shipping_tags', List(String)),
        ('shipping_free_shipping', Boolean),
        ('shipping_mode', String),
        ('shipping_dimensions', String),
        ('rule_value', Null),
        ('rule_free_mode', String),
        ('id', Int64),
        ('nmp_pm_description', String),
        ('nmp_pm_id', String),
        ('nmp_pm_type', String),
        ('seller_id', Int64),
        ('var_comb_value_id', String),
        ('var_comb_name', String),
        ('var_comb_value_name', String),
        ('var_comb_id', String),
        ('var_seller_custom_field', String),
        ('var_picture_ids', List(String)),
        (

In [8]:
df_product_dup = df.filter(pl.col('product_id').is_duplicated())
print('Filas duplicadas en product_id:')
print(len(df_product_dup))

Filas duplicadas en product_id:
1992099


In [9]:
# Agrupar por todas las columnas
duplicados_completos = (
    df.group_by(df.columns)
      .agg(pl.len().alias('count'))
      .filter(pl.col('count') > 1)
)
print('Registros duplicados (completos):')
print(len(duplicados_completos))

Registros duplicados (completos):
6440


In [11]:
df.select('product_id').sample(3, seed=0)

product_id
str
"""MLA5231674569"""
"""MLA4220868420"""
"""MLA6475015600"""


In [12]:
df.filter(pl.col('product_id') == 'MLA4220868420').select('condition', 'price', 'title')

condition,price,title
str,f64,str
"""new""",100.0,"""Sjcam Sj4000 Wifi Sj 5000 Bate…"
"""new""",100.0,"""Sjcam Sj4000 Wifi Sj 5000 Bate…"
"""new""",100.0,"""Sjcam Sj4000 Wifi Sj 5000 Bate…"
"""new""",100.0,"""Sjcam Sj4000 Wifi Sj 5000 Bate…"
"""new""",100.0,"""Sjcam Sj4000 Wifi Sj 5000 Bate…"
…,…,…
"""new""",100.0,"""Sjcam Sj4000 Wifi Sj 5000 Bate…"
"""new""",100.0,"""Sjcam Sj4000 Wifi Sj 5000 Bate…"
"""new""",100.0,"""Sjcam Sj4000 Wifi Sj 5000 Bate…"


In [15]:
df.select([col for col in df.columns if 'var' in col]).sample(10, seed=25)

var_comb_value_id,var_comb_name,var_comb_value_name,var_comb_id,var_seller_custom_field,var_picture_ids,var_sold_quantity,var_available_quantity,var_id,var_price
str,str,str,str,str,list[str],i64,i64,i64,f64
"""92018""","""Color Primario""","""Lavanda""","""83000""",null,"[""397701-MLA20405992799_092015"", ""778801-MLA20405994047_092015"", ""348801-MLA20405994049_092015""]",0,2,9450267851,194.99
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
"""82026""","""Color Secundario""","""Rojo""","""73001""",null,"[""824101-MLA20273468733_042015"", ""546101-MLA20273468746_042015"", … ""476101-MLA20273468449_042015""]",0,1,9292104817,45.0
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
"""132011""","""Talle""","""31""","""73002""","""SSCWH""","[""666601-MLA20355727500_072015"", ""485501-MLA20355727366_072015"", … ""955801-MLA20420797450_092015""]",0,5,9616250910,749.0
null,null,null,null,null,null,null,null,null,null


In [14]:
# Columnas llamadas 'id'
df.select([col for col in df.columns if 'id' in col]).sample(10, seed=25)

deal_ids,id,nmp_pm_id,seller_id,var_comb_value_id,var_comb_id,var_picture_ids,var_id,site_id,listing_type_id,attr_value_id,attr_attribute_group_id,attr_id,parent_item_id,category_id,pic_id,product_id,official_store_id,currency_id,video_id,catalog_product_id
str,i64,str,i64,str,str,list[str],i64,str,str,str,str,str,str,str,str,str,f64,str,str,f64
"""""",null,"""MLATB""",4917621612,"""92018""","""83000""","[""397701-MLA20405992799_092015"", ""778801-MLA20405994047_092015"", ""348801-MLA20405994049_092015""]",9450267851,"""MLA""","""bronze""","""Season-Autumn-Winter""","""FIND""","""Season""",null,"""MLA64653""","""498701-MLA6655124735_092015""","""MLA6655124735""",null,"""ARS""",null,null
"""""",null,"""MLAMO""",4101959504,null,null,null,null,"""MLA""","""bronze""",null,null,null,"""MLA8069337522""","""MLA41287""","""18937-MLA2710917485_092014""","""MLA2710917485""",null,"""ARS""",null,null
"""""",null,"""MLATB""",4183308329,null,null,null,null,"""MLA""","""bronze""",null,null,null,"""MLA1240644124""","""MLA4785""","""120101-MLA2218382955_032015""","""MLA2218382955""",null,"""ARS""",null,null
"""""",null,"""MLATB""",3884593281,"""82026""","""73001""","[""824101-MLA20273468733_042015"", ""546101-MLA20273468746_042015"", … ""476101-MLA20273468449_042015""]",9292104817,"""MLA""","""bronze""","""Season-Spring-Summer""","""FIND""","""Season""","""MLA3547762102""","""MLA26536""","""476101-MLA1932237478_042015""","""MLA1932237478""",null,"""ARS""",null,null
"""""",null,null,2399979535,null,null,null,null,"""MLA""","""gold_premium""","""ESTACIOINCLU-Y""","""COMODIDADES""","""ESTACIOINCLU""","""MLA4016986071""","""MLA50279""","""8058-MLA8794783709_112013""","""MLA8794783709""",null,"""ARS""",null,null
"""""",null,null,7926648350,null,null,null,null,"""MLA""","""free""","""MLA1744-LLANALEAC-N""","""EXTERIOR""","""MLA1744-LLANALEAC""",null,"""MLA6628""","""228801-MLA3996399802_092015""","""MLA3996399802""",null,"""ARS""",null,null
"""""",null,null,8926326574,null,null,null,null,"""MLA""","""gold_premium""","""MLA1744-MARC-CHEVROLET""","""FIND""","""MLA1744-MARC""","""MLA9319455266""","""MLA3188""","""104401-MLA7812805402_062015""","""MLA7812805402""",null,"""ARS""",null,null
"""UQU1U""",73328,null,6879780273,"""132011""","""73002""","[""666601-MLA20355727500_072015"", ""485501-MLA20355727366_072015"", … ""955801-MLA20420797450_092015""]",9616250910,"""MLA""","""gold_special""","""Season-Autumn-Winter""","""FIND""","""Season""","""MLA3151741240""","""MLA117507""","""509901-MLA1470121909_102015""","""MLA1470121909""",null,"""ARS""",null,null
"""""",null,null,1594358977,null,null,null,null,"""MLA""","""gold""","""MLA1744-ABS-Y""","""SECURITY""","""MLA1744-ABS""",null,"""MLA30846""","""255901-MLA8271369784_092015""","""MLA8271369784""",null,"""ARS""",null,null


In [16]:
# Conteo de productos únicos
df.group_by('product_id').agg(pl.count('product_id').alias('count')).sort('count', descending=True)

product_id,count
str,u32
"""MLA6356328214""",16128
"""MLA6655124735""",9072
"""MLA6870343634""",9072
"""MLA2618837555""",7776
"""MLA6475015600""",7392
…,…
"""MLA3009003369""",1
"""MLA6360218686""",1
"""MLA2867543668""",1


In [18]:
df.filter(pl.col('product_id') == 'MLA6356328214').select('condition')

condition
str
"""new"""
"""new"""
"""new"""
"""new"""
"""new"""
…
"""new"""
"""new"""
"""new"""


In [19]:
# Columnas de tipo list[str]
df.select([col for col, dtype in df.schema.items() if dtype == pl.List])

shipping_methods,shipping_tags,var_picture_ids,coverage_areas
list[null],list[str],list[str],list[null]
[],[],null,[]
[],[],null,[]
[],[],null,[]
[],[],null,[]
[],[],null,[]
…,…,…,…
[],[],null,[]
[],[],null,[]
[],[],null,[]


In [20]:
# Columnas llamadas 'payment'
df.select([col for col in df.columns if 'attr' in col])

attr_value_id,attr_attribute_group_id,attr_name,attr_value_name,attr_attribute_group_name,attr_id
str,str,str,str,str,str
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
…,…,…,…,…,…
null,null,null,null,null,null
null,null,null,null,null,null
null,null,null,null,null,null
